# Install detectron2

In [0]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-0ua9bje2
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-0ua9bje2
     |████████████████████████████████| 276kB 6.4MB/s 
  Created wheel for fvcore: filename=fvcore-0.1-cp36-none-any.whl size=44465 sha256=89129e6be2b231e85542f5f51cb428bee525714f1b13eed3f02956ab83d2e78a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ystfo1ji/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=e865d98a6924339e5496c6403419231414bf2d507c95ad3446c12233976658a5
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully b

'1.5.0+cu101'

In [0]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 4689 (delta 9), reused 18 (delta 7), pack-reused 4661
Receiving objects: 100% (4689/4689), 2.38 MiB | 28.67 MiB/s, done.
Resolving deltas: 100% (3322/3322), done.
Obtaining file:///content/detectron2_repo
  Running setup.py develop for detectron2


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import json
import numpy as np
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from google.colab import drive
import os
drive.mount('/content/drive')
DatasetCatalog.clear()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from detectron2.data.datasets import register_coco_instances
# datasets' unique names to register
train = "pets_train"
test = "pets_test"

image_root = "/content/drive/My Drive/VietAnh/dataset/image"
instances_json = "/content/drive/My Drive/VietAnh/dataset/annotations/data_seg_merged/train_pets.json"
instances_test_json = "/content/drive/My Drive/VietAnh/dataset/annotations/data_seg_merged/test_pets.json"

register_coco_instances(train, {}, instances_json, image_root)
register_coco_instances(test, {}, instances_test_json, image_root)

In [0]:
meta_train = MetadataCatalog.get(train)
dicts_train = DatasetCatalog.get(train)


[05/09 17:17:01 d2.data.datasets.coco]: Loading /content/drive/My Drive/VietAnh/dataset/annotations/data_seg_merged/train_pets.json takes 2.07 seconds.
WARNING [05/09 17:17:01 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/09 17:17:01 d2.data.datasets.coco]: Loaded 2937 images in COCO format from /content/drive/My Drive/VietAnh/dataset/annotations/data_seg_merged/train_pets.json


In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

# set hyper parameters
cfg = get_cfg()
cfg.merge_from_file("/content/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.DATASETS.TRAIN = (train,)
cfg.DATASETS.TEST = (test, )   
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 5000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.OUTPUT_DIR = "weights/resnext101"
cfg.INPUT.MASK_FORMAT = 'bitmask'

# training
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[05/09 17:24:32 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

X-101-32x8d.pkl: 356MB [00:05, 67.8MB/s]                           


[05/09 17:24:39 d2.checkpoint.c2_model_loading]: Remapping C2 weights ......
[05/09 17:24:39 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.bias            loaded from res2_0_branch2a_bn_b              of shape (256,)
[05/09 17:24:39 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.running_mean    loaded from res2_0_branch2a_bn_rm             of shape (256,)
[05/09 17:24:39 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.running_var     loaded from res2_0_branch2a_bn_riv            of shape (256,)
[05/09 17:24:39 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.weight          loaded from res2_0_branch2a_bn_s              of shape (256,)
[05/09 17:24:39 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.weight               loaded from res2_0_branch2a_w                 of shape (256, 64, 1, 1)
[05/09 17:24:39 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv2.norm.bias    

/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.
/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


WARNING [05/09 17:24:54 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/09 17:24:54 d2.data.datasets.coco]: Loaded 734 images in COCO format from /content/drive/My Drive/VietAnh/dataset/annotations/data_seg_merged/test_pets.json
[05/09 17:24:54 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|    cat     | 247          |    dog     | 486          |
|            |              |            |              |
|   total    | 733          |            |              |
[05/09 17:24:54 d2.data.common]: Serializing 734 elements to byte tensors and concatenating them all ...
[05/09 17:24:54 d2.data.common]: Serialized dataset takes 0.94 MiB
WARNING [05/09 17:24:54 d2.engine.defaults]: No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.
[05/09 17